In [10]:
import bs4 as bs
from urllib.request import urlopen
import datetime
import time
import csv
import pandas as pd

start_time = time.time()

def scrape_and_get_data(url,page):
    #print(f'page: {page}')
    source = urlopen(url).read()
    global soup
    soup = bs.BeautifulSoup(source, 'lxml')
    for i in soup.find_all('div', {'class': 'product-row'}):
        #print(i.find('h2', {'class': 'product-name'}).text.strip())
        name.append(i.find('h2', {'class': 'product-name'}).text.strip())
        brand.append(i.find('a', {'class': 'product-brand'}).text.strip())
        category.append(i.find('p', {'class': 'product-category'}).text.strip())
        reviews.append(i.find('div', {'class': 'stars-rating'}).text.strip())
        rating.append(i.find('a', {'class': 'js-save-keyword js-scroll-by-hash'}).get('title'))
        for j in i.find_all('div', {'class': 'attributes-row'}):
            attribute = j.find('span', {'class': 'attribute-name'}).text.strip()
            value = j.find('span', {'class': 'attribute-value'}).text.strip()
            if attribute == 'Ekran':
                screen.append(value)
            elif attribute == 'Procesor':
                processor.append(value)
            elif attribute == 'Pamięć':
                memory.append(value)
            elif attribute == 'Grafika':
                graphics_card.append(value)                
            elif attribute == 'Dysk':
                disk.append(value)
            elif attribute == 'System operacyjny':
                operating_system.append(value)
        #getting price
        try:
            current_price.append(i.find('div', {'class': 'price-normal selenium-price-normal'}).text.strip())
        except AttributeError:
            current_price.append(i.find('div', {'class': 'price-normal'}).text.strip())            
        # getting old price
        try:
            former_price.append(i.find('div', {'class': 'price-old'}).text.strip())
        except AttributeError:
            former_price.append('')
        # getting delivery type
        if i.find('span', {'class': 'delivery-message-label'}).text.strip() == 'Sprawdź dostępność w sklepach':
            delivery.append('store only')
        else:
            delivery.append('online')
        #page number
        item_page.append(page)
        

#lists to store scraped data, will be used as columns
name, brand, category, reviews, rating, screen, processor, memory = ([] for i in range(8))
graphics_card, disk, operating_system, current_price, former_price, delivery, item_page = ([] for i in range(7))

#first page
main_page = 'https://www.euro.com.pl/laptopy-i-netbooki.bhtml'

#scraping data from first page
scrape_and_get_data(main_page,1)

#Accessing scraped first page to get the number of pages and scrape thrugh them
pages = [int(i.text.strip()) for i in soup.find_all('a', {'class': 'paging-number'})]
max_page = max(pages)
max_page        

#scraping pages from 2 to last
for page in range(2,max_page+1):
    page_address = f'https://www.euro.com.pl/laptopy-i-netbooki,strona-{page}.bhtml'
    scrape_and_get_data(page_address,page)
    

#new column with date
date_now = [datetime.datetime.now().strftime("%m/%d/%Y %H:%M") for i in range(len(name))]        

#zipping all the lists together
base_lists = zip(date_now, name, brand, category, reviews, rating, screen, processor
                     , memory, graphics_card, disk, operating_system, current_price, former_price
                     , delivery, item_page)

#saving all the data to a csv file
with open('laptops_scraped.csv', 'a+', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    #writer.writerow(col_names)
    writer.writerows(base_lists)
    
print("Script executed in %s seconds" % (time.time() - start_time))

Script executed in 285.411554813385 seconds


In [13]:
col_names = ['Time', 'Name', 'Brand', 'Category', 'Reviews', 'Rating', 'Screen', 'Processor'
                     , 'Memory', 'Graphics_Card', 'Disk', 'Operating_System', 'Current_Price', 'Former_Price'
            , 'Delivery', 'Item_Page']

df = pd.read_csv('laptops_scraped.csv',
            names=col_names)

df.head()


,Time,Name,Brand,Category,Reviews,Rating,Screen,Processor,Memory,Graphics_Card,Disk,Operating_System,Current_Price,Former_Price,Delivery,Item_Page
0,08/30/2021 09:32,"Acer Aspire 5 A515-56-55YP 15,6"" Intel® Core™ ...",Acer,laptop,(11 opinii),"4.5 na 5, przeczytaj 11 opinii o Aspire 5 A515...","15,6 cala, 1920 x 1080 pikseli","Intel® Core™ i5 11gen 1135G7 2,4 - 4,2 GHz",8 GB RAM,Intel® Iris Xe Graphics,512 GB SSD,bez systemu,2 399 zł,2 799 zł,online,1
1,08/30/2021 09:32,"HP 14s-dq2009nw 14"" Intel® Core™ i3-1115G4 - 8...",HP,laptop,(27 opinii),"5 na 5, przeczytaj 27 opinii o 14s-dq2009nw 14...","14 cali, 1920 x 1080 pikseli","Intel® Core™ i3 11gen 1115G4 3,0 - 4,1 GHz",8 GB RAM,Intel® UHD Graphics,256 GB SSD,Windows 10 S,1 999 zł,NaN,online,1
2,08/30/2021 09:32,"Acer Nitro 5 AN515-45-R7LR 15,6"" AMD Ryzen 5 5...",Acer,laptop,NaN,"Brak opinii o Nitro 5 AN515-45-R7LR 15,6"" AMD ...","15,6 cala, 1920 x 1080 pikseli","AMD Ryzen™ 5 5600H 3,3 - 4,2 GHz",16 GB RAM,NVIDIA® GeForce RTX™ 3060 + AMD Radeon™ Graphics,512 GB SSD,Windows 10 Home Edition,5 199 zł,5 499 zł,online,1
3,08/30/2021 09:32,"Acer Aspire 3 A315-23-R607 15,6"" AMD Ryzen 3 3...",Acer,laptop,NaN,"Brak opinii o Aspire 3 A315-23-R607 15,6"" AMD ...","15,6 cala, 1920 x 1080 pikseli","AMD Ryzen™ 3 3250U 2,6 - 3,5 GHz",4 GB RAM,AMD Radeon™ Graphics,256 GB SSD,bez systemu,1 549 zł,1 799 zł,online,1
4,08/30/2021 09:32,"ASUS VivoBook 17 F712JA-BX351T 17,3"" Intel® Co...",ASUS,laptop,NaN,"Brak opinii o VivoBook 17 F712JA-BX351T 17,3"" ...","17,3 cala, 1600 x 900 pikseli","Intel® Core™ i3 10gen 1005G1 1,2 - 3,4 GHz",8 GB RAM,Intel® UHD Graphics,512 GB SSD,Windows 10 Home Edition,2 399 zł,2 699 zł,online,1
